In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd

train = pd.read_csv('atis_intents_train.csv',header=None)
test = pd.read_csv('atis_intents_test.csv',header=None)
# Load the BERT tokenizer and model



In [2]:
train


,0,1
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...
...,...,...
4829,atis_airfare,what is the airfare for flights from denver t...
4830,atis_flight,do you have any flights from denver to baltim...
4831,atis_airline,which airlines fly into and out of denver
4832,atis_flight,does continental fly from boston to san franc...


In [3]:
from sklearn.preprocessing import LabelEncoder as LE
le = LE()
le.fit(train[0])

LabelEncoder()

In [4]:
train_y = le.transform(train[0])
train_x = train[1]

test_y = le.transform(test[0])
test_x = test[1]

In [5]:
train_y

array([4, 4, 5, ..., 3, 4, 4])

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)


inputs = tokenizer(list(train_x[0:100]), padding=True, truncation=True, return_tensors="pt")
labels = torch.tensor(train_y[0:100])



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
len(inputs)

3

In [13]:
from tqdm import tqdm
for j in tqdm(range(0,len(train_x),100)):
  inputs = tokenizer(list(train_x[j:j+100]), padding=True, truncation=True, return_tensors="pt")
  labels = torch.tensor(train_y[j:j+100])
  
  optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
  for epoch in range(3):
      outputs = model(**inputs, labels=labels)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

# Evaluate the model
test_texts = list(test_x)
test_labels = test_y



100%|██████████| 49/49 [30:27<00:00, 37.29s/it]


In [14]:
test_inputs = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")
test_labels = torch.tensor(test_labels)

with torch.no_grad():
    logits = model(**test_inputs).logits
    predictions = torch.argmax(logits, dim=-1)
    accuracy = (predictions == test_labels).float().mean()

print("Accuracy:", accuracy.item())

Accuracy: 0.9862499833106995


In [15]:
predictions

tensor([4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 2, 3, 2, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 3, 4, 4, 4, 4, 4, 4, 6, 4, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 7, 3, 3, 4, 4, 4, 4, 4, 2, 2, 4, 0, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 2, 1, 4, 2, 4, 4, 4, 4, 4, 3,
        3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4,
        4, 4, 4, 2, 0, 4, 4, 4, 4, 4, 4, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 3, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 2, 2, 2, 2, 6, 4, 6, 3, 2, 4, 4,
        4, 4, 4, 4, 4, 4, 0, 4, 4, 6, 4,

In [17]:
from sklearn.metrics import classification_report

print(classification_report(test_labels,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      0.67      0.80         9
           2       0.96      1.00      0.98        48
           3       0.90      1.00      0.95        38
           4       1.00      0.99      0.99       632
           5       0.00      0.00      0.00         1
           6       1.00      1.00      1.00        36
           7       0.25      0.33      0.29         3

    accuracy                           0.99       800
   macro avg       0.76      0.75      0.75       800
weighted avg       0.99      0.99      0.99       800



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
